In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import json
import os
from tqdm import tqdm
from utilities import create_file_list

data_dir = '/home/tunx404/Miscellaneous/data/'

In [2]:
def imshow(image):
    # print(f'Shape = {image.shape}, min value = {np.min(image)}, max value = {np.max(image)}')
    image = np.reshape(image, (image.shape[0], image.shape[1], -1))
    plt.figure(figsize=(20, 10))
    if image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image, vmin=0)
    else:
        plt.imshow(image, cmap='gray', vmin=0)
    plt.show()
    
def constraint_range_01(image):
    image = (image - np.min(image))/(np.max(image) - np.min(image))
    return image

def constraint_min_0(image):
    image = (image - np.min(image))
    return image

def constraint_cut_0(image):
    image[image < 0] = 0
    return image

In [19]:
def add_rain(city_name, image_name, rain_level, rain_pattern, alpha=0.01, beta=0.005, a=0.01, show_disparity=False, show_depth_raw=False, show_depth=False, show_pattern=False, show_rain_intensity=False, show_rain=False, show_fog=False, show_original=False, show_output=True, show_sample=False):    
    city_image_name = city_name + '/' + image_name
    camera_calibration_path = data_dir + 'Cityscapes/camera_trainvaltest/camera/train/' + city_image_name + '_camera.json'
    camera_calibration_file = open(camera_calibration_path)
    camera_calibration = json.load(camera_calibration_file)
    camera_calibration_file.close()
    baseline = camera_calibration['extrinsic']['baseline']
    fxy = (camera_calibration['intrinsic']['fx'] + camera_calibration['intrinsic']['fy'])/2

    image_path = data_dir + 'Cityscapes/leftImg8bit_trainvaltest/leftImg8bit/train/' + city_image_name + '_leftImg8bit.png'
    image = cv2.imread(image_path)
    
    rain_pattern_path = '/home/tunx404/Miscellaneous/18744/rain-rendering/data/output/cityscapes/leftImg8bit/train/' + city_name + '/rain/' + rain_level + '/rain_mask/' + rain_pattern + '_leftImg8bit.png'
    R_pattern = cv2.imread(rain_pattern_path)
    R_pattern = cv2.resize(R_pattern, (2048, 1024))
    R_pattern = np.average(R_pattern, axis=2)
    R_pattern = constraint_min_0(R_pattern)/255
    # R_pattern = constraint_range_01(R_pattern)
    R_pattern = R_pattern.astype(np.float32)
    R_pattern = cv2.ximgproc.guidedFilter(image, R_pattern, 3, 1e-3)
    # R_pattern = constraint_range_01(R_pattern)
    R_pattern = constraint_cut_0(R_pattern)

    disparity_path = data_dir + 'Cityscapes/disparity_trainvaltest/disparity/train/' + city_image_name + '_disparity.png'
    disparity = cv2.imread(disparity_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
    disparity[disparity > 0] = (disparity[disparity > 0] - 1)/256
    if show_disparity:
        print('Disparity')
        imshow(disparity)

#     depth = baseline*fxy/disparity
#     # depth[depth == np.Inf] = 0
#     depth[depth == np.Inf] = np.average(depth[depth != np.Inf])
    
#     if show_depth_raw:
#         print('Raw depth map')
#         imshow(depth)
    
#     depth_inverted_rain_path = 'data/RainCityscapes/leftImg8bit_trainval_rain/depth_rain/train/' + city_image_name + '_000019_depth_rain.png'
#     depth_inverted_rain = cv2.imread(depth_inverted_rain_path).astype(np.float32)
#     depth_rain = 255 - np.average(depth_inverted_rain, axis=2)
    
    depth_refined_path = data_dir + 'Foggy Cityscapes/depth_stereoscopic_trainvaltest/depth_stereoscopic/train/' + city_image_name + '_depth_stereoscopic.mat'
    depth_refined = scipy.io.loadmat(depth_refined_path)
    depth_refined = depth_refined['depth_map'].astype(np.float32)
    
    depth = depth_refined

    A0 = 1.0
    d1 = 2*fxy*a

    # (4)
    A = 1 - np.exp(-beta*depth)
    A_guided = cv2.ximgproc.guidedFilter(image, A, 10, 1e-3)
    A_guided = constraint_cut_0(A_guided)
    A_guided = constraint_range_01(A_guided)
    
    # (3)
    t_r = np.exp(-alpha*np.maximum(d1, depth))
    t_r_guided = cv2.ximgproc.guidedFilter(image, t_r, 10, 1e-3)
    t_r_guided = constraint_cut_0(t_r_guided)
    t_r_guided = constraint_range_01(t_r_guided)

    # (2)
    R = np.multiply(R_pattern, t_r_guided)
    
    if show_depth:
        print('Refined depth map')
        imshow(depth)
        imshow(np.log(depth))
        
    if show_rain_intensity:
        print('Rain streaks intensity')
        # imshow(t_r)
        imshow(t_r_guided)
    
    if show_pattern:
        print('Rain pattern')
        imshow(R_pattern)

    if show_rain:
        print('Rain layer')
        imshow(R)
        
    if show_fog and show_depth:
        print('Depth map')
        imshow(depth)
        
    if show_fog:
        print('Fog layer')
        # imshow(A)
        imshow(A_guided)

    # (1)
    output = np.zeros_like(image)
    for c in range(output.shape[2]):
        output[:, :, c] = image[:, :, c]*(1 - R - A_guided) + 255*(R + A0*A_guided)

    if show_original:
        print('Original')
        imshow(image)
        
    if show_output:
        print('Output')
        imshow(output)
    
    # if show_sample:
    #     output_sample_path = data_dir + 'RainCityscapes/leftImg8bit_trainval_rain/leftImg8bit_rain/train/' + image_name + '_000019_leftImg8bit_rain_alpha_0.01_beta_0.005_dropsize_0.01_pattern_6.png'
    #     output_sample = cv2.imread(output_sample_path)
    #     print('Sample')
    #     imshow(output_sample)
        
    return output

In [20]:
data_cityscapes_dir = data_dir + 'Cityscapes/leftImg8bit_trainvaltest/leftImg8bit/train'
city_name_list, city_path_list = create_file_list(data_cityscapes_dir)

print(city_name_list)
print(city_path_list)

# %cd data/output_rain
# for city_name in city_name_list:
#     !mkdir $city_name

['aachen', 'bochum', 'bremen', 'cologne', 'darmstadt', 'dusseldorf', 'erfurt', 'hamburg', 'hanover', 'jena', 'krefeld', 'monchengladbach', 'strasbourg', 'stuttgart', 'tubingen', 'ulm', 'weimar', 'zurich']
['aachen', 'bochum', 'bremen', 'cologne', 'darmstadt', 'dusseldorf', 'erfurt', 'hamburg', 'hanover', 'jena', 'krefeld', 'monchengladbach', 'strasbourg', 'stuttgart', 'tubingen', 'ulm', 'weimar', 'zurich']


In [32]:
for c in range(len(city_name_list)):
    city_name = city_name_list[c]
    city_path = os.path.join(data_cityscapes_dir, city_path_list[c])
    
    print(c)
    print(city_name)
    print(city_path)

    image_name_list, _ = create_file_list(city_path)
    for i in range(len(image_name_list)):
        temp = image_name_list[i].split('_')
        image_name_list[i] = temp[0] + '_' + temp[1] + '_' + temp[2]
    # print(image_name_list) 

    output_dir = '/home/tunx404/Miscellaneous/data/RainAddition/train/'
    for i in tqdm(range(len(image_name_list))):
        city_name  = city_name
        image_name = image_name_list[i]

        rain_pattern = image_name
        type_list = ['H', 'M', 'S']
        rain_level_list = ['20mm', '5mm', '2mm']
        alpha_list = [1e-2, 2e-3, 1e-4]
        beta_list  = [1e-2, 2e-3, 1e-4]
        a = 0.1

        for (type, rain_level, alpha, beta) in list(zip(type_list, rain_level_list, alpha_list, beta_list)):
            output_file_name = output_dir + city_name + '/' + image_name + '_' + type + '.png'
            if not os.path.exists(output_file_name):
                print(output_file_name)
                output = add_rain(city_name, image_name, rain_level, rain_pattern, alpha, beta, a, show_output=False)
                cv2.imwrite(output_file_name, output)

0
aachen
/home/tunx404/Miscellaneous/data/Cityscapes/leftImg8bit_trainvaltest/leftImg8bit/train/aachen


100%|████████████████████████████████████████████████████| 174/174 [00:00<00:00, 88784.54it/s]


1
bochum
/home/tunx404/Miscellaneous/data/Cityscapes/leftImg8bit_trainvaltest/leftImg8bit/train/bochum


100%|██████████████████████████████████████████████████████| 96/96 [00:00<00:00, 71863.86it/s]


2
bremen
/home/tunx404/Miscellaneous/data/Cityscapes/leftImg8bit_trainvaltest/leftImg8bit/train/bremen


...

100%|██████████████████████████████████████████████████████| 95/95 [00:00<00:00, 62055.58it/s]


16
weimar
/home/tunx404/Miscellaneous/data/Cityscapes/leftImg8bit_trainvaltest/leftImg8bit/train/weimar


100%|████████████████████████████████████████████████████| 142/142 [00:00<00:00, 80279.17it/s]


17
zurich
/home/tunx404/Miscellaneous/data/Cityscapes/leftImg8bit_trainvaltest/leftImg8bit/train/zurich


100%|████████████████████████████████████████████████████| 122/122 [00:00<00:00, 83069.01it/s]


<!-- # Coefficient: alpha -->